In [ ]:
# Google Earth Engine packages
import ee
import geemap

import numpy as np

In [ ]:
# initialize GEE at the beginning of session
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()         # authenticate when using GEE for the first time
    ee.Initialize()

In [ ]:
import configparser
import ast

# read local config.ini file
config = configparser.ConfigParser()
config.read('config.ini')

# get file config from config.ini
#dir_input = config['FILE_SETTINGS']['DIR_INPUT']
dir_output = config['FILE_SETTINGS']['DIR_OUTPUT']
output_gpkg = dir_output + config['FILE_SETTINGS']['GPKG_NAME']

# get date range for forcing data
#date_range = ast.literal_eval(config['CONFIG']['DATE_RANGE'])

In [ ]:
import geopandas as gpd

catchment_new = gpd.read_file(output_gpkg, layer='catchment_new')
catchment = geemap.geopandas_to_ee(catchment_new)

***

In [ ]:
def renameBandName(b):
    split = ee.String(b).split('_')   
    #return ee.String(split.get(0)).cat('_').cat(ee.String(split.get(1)))
    return ee.String(split.splice(split.length().subtract(2),1).join("_"))


def buildFeature(i):
    t1 = startDate.advance(i,'day')
    t2 = t1.advance(1,'day')
    #feature = ee.Feature(point)
    dailyColl = collection.filterDate(t1, t2)
    dailyImg = dailyColl.toBands()
    # renaming and handling names
    bands = dailyImg.bandNames()
    renamed = bands.map(renameBandName)
    # Daily extraction and adding time information
    dict = dailyImg.rename(renamed).reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=catchment,
    ).combine(
      ee.Dictionary({'system:time_start':t1.millis(),'isodate':t1.format('YYYY-MM-dd')})
    )
    return ee.Feature(None,dict)

In [ ]:
startDate = ee.Date('2020-01-01')
endDate = ee.Date('2030-01-31')

In [ ]:
collection = ee.ImageCollection('NASA/GDDP-CMIP6')\
    .select('tas')\
    .filterDate(startDate, endDate)\
    .filterBounds(catchment)\
    .filter(ee.Filter.eq('model', 'ACCESS-CM2'))

In [ ]:
n = endDate.difference(startDate,'day').subtract(1)
fc1 = ee.FeatureCollection(ee.List.sequence(0,n).map(buildFeature))

In [ ]:
%%time

downloadUrl = fc1.getDownloadURL(filetype='csv',filename='CMIP6')
print('URL for downloading FeatureCollection as CSV', downloadUrl)

In [ ]:
%%time

# Export the FeatureCollection to a KML file.
task1 = ee.batch.Export.table.toDrive(**{
  'collection': fc1,
  'description':'CMIP6_tas',
  'fileFormat': 'CSV'
})
task1.start()

while task1.active():
    print(".", end = '')
    
print('done.')

In [ ]:
fc1